# Simple Text2Cypher Example

This notebook demonstrates how to use the simplified Text2Cypher workflow to convert natural language questions into Cypher queries for Neo4j.

In [ ]:
import sys
import os

# Add the project root to Python path
sys.path.append("..")

In [ ]:
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI

from simple_text2cypher.retrievers.cypher_examples import YAMLCypherExampleRetriever
from simple_text2cypher.workflows.simple_text2cypher_workflow import create_simple_text2cypher_workflow

In [ ]:
# Load environment variables
load_dotenv()

## Initialize Components

Set up the Neo4j graph connection, LLM, and Cypher example retriever.

In [ ]:
# Initialize Neo4j graph connection
graph = Neo4jGraph(enhanced_schema=True)
print("Connected to Neo4j")
print(f"Schema: {graph.schema[:500]}...")  # Print first 500 chars of schema

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("LLM initialized")

In [ ]:
# Initialize Cypher example retriever
cypher_query_yaml_file_path = "../data/example/queries.yml"
cypher_example_retriever = YAMLCypherExampleRetriever(
    cypher_query_yaml_file_path=cypher_query_yaml_file_path
)
print(f"Cypher examples loaded from: {cypher_query_yaml_file_path}")

## Create the Workflow

Set up the simple text2cypher workflow with guardrails and validation.

In [ ]:
# Define the scope of your application
scope_description = "This application answers questions about your graph database."

# Create the workflow
agent = create_simple_text2cypher_workflow(
    llm=llm,
    graph=graph,
    scope_description=scope_description,
    cypher_example_retriever=cypher_example_retriever,
    llm_cypher_validation=False,  # Use rule-based validation for speed
    max_attempts=3,
    attempt_cypher_execution_on_final_attempt=True,
)

print("Text2Cypher workflow created successfully!")

## Visualize the Workflow

Display the workflow graph to understand the execution flow.

In [ ]:
from IPython.display import Image, display

try:
    # Display the workflow graph
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Workflow structure:")
    print("Input → Guardrails → Planner → Text2Cypher → Gather Results → Summarize → Validate → Final Answer")

## Example Usage

Run some example questions through the workflow.

In [ ]:
# Example question
question = "What nodes exist in this graph?"

response = await agent.ainvoke({
    "question": question,
    "data": [],
    "history": []
})

print(f"Question: {response.get('question')}")
print(f"\nAnswer: {response.get('answer')}")

## Examine the Results

Look at the generated Cypher queries and execution steps.

In [ ]:
# Display the execution steps
print("Execution Steps:")
for step in response.get("steps", []):
    print(f"  - {step}")

In [ ]:
# Display generated Cypher queries and results
cyphers = response.get("cyphers", [])
print(f"\nGenerated {len(cyphers)} Cypher queries:")

for i, cypher_result in enumerate(cyphers):
    print(f"\n--- Query {i+1} ---")
    print(f"Task: {cypher_result.get('task')}")
    print(f"Cypher: {cypher_result.get('statement')}")
    
    records = cypher_result.get('records', [])
    print(f"Results: {len(records)} records")
    
    # Show first few records
    for j, record in enumerate(records[:3]):
        print(f"  Record {j+1}: {record}")
    
    if len(records) > 3:
        print(f"  ... and {len(records) - 3} more records")

## Try Your Own Questions

Modify the question below to test with your own data.

In [ ]:
# Try your own question here
my_question = "How many relationships are in this graph?"

my_response = await agent.ainvoke({
    "question": my_question,
    "data": [],
    "history": []
})

print(f"Question: {my_response.get('question')}")
print(f"\nAnswer: {my_response.get('answer')}")

# Show Cypher queries
my_cyphers = my_response.get("cyphers", [])
for cypher_result in my_cyphers:
    print(f"\nCypher: {cypher_result.get('statement')}")
    records = cypher_result.get('records', [])
    if records:
        print(f"Result: {records[0] if records else 'No results'}")

## Summary

This example demonstrates:

1. **Setup**: Initialize Neo4j connection, LLM, and Cypher examples
2. **Workflow Creation**: Build the text2cypher workflow with guardrails
3. **Question Processing**: Convert natural language to Cypher and execute
4. **Results**: Examine generated queries and returned data

The workflow includes:
- **Guardrails**: Validate questions are in scope
- **Planning**: Break down complex questions
- **Text2Cypher**: Generate, validate, and execute Cypher
- **Summarization**: Format results in natural language
- **Validation**: Ensure answer quality

For production use, consider:
- Adding more comprehensive Cypher examples
- Customizing the scope description
- Enabling LLM-based validation for higher accuracy
- Implementing conversation history for multi-turn interactions